In [11]:
import pandas 

kinetics = pandas.read_csv( 'kinetics.csv' )
thermal = pandas.read_csv( 'thermal.csv' ) 

print( kinetics.columns )
print( thermal.columns ) 

Index(['well', 'rate', 'mutant', 'protein_yield', 'dilution', 'note'], dtype='object')
Index(['well', 'rate', 'mutant'], dtype='object')


In [12]:
# for thermal 
# convert well indexes into temperatures based on map 

temp_key = pandas.read_csv( '../raw/temperature_key.csv', index_col='Well' )
thermal['temp'] = thermal.well.str[0].map( temp_key.Celsius.to_dict() ) 

print( thermal.sample( 3 ) )

    well     rate mutant   temp
890  G10  0.25100  M358T  32.90
537   F8 -0.00023  H119N  35.75
130   C8  0.00027  E154D  44.30


In [13]:
# for kinetics, convert the rate into units (1/min) 
# hard-coded values:
#  0.0002 is the conversion from OD to M
#  0.25 is because we dilute the protein 4-fold upon initiation 
#  113330 is the extinction coef of BglB, used to convert mg/mL to M

kinetics[ 'kobs' ] = kinetics['rate'] * 0.0002 / ( kinetics[ 'protein_yield' ] * kinetics[ 'dilution' ] * 0.25 / 113300 )

print( kinetics.sample( 3 ) ) 

    well    rate mutant  protein_yield  dilution  note        kobs
98    C1  0.0968  L362M           1.99      0.01   NaN  440.902111
100   E1  0.0122  L362M           1.99      0.01   NaN   55.568241
185  B12  0.0185  N220G           2.18      0.01   NaN   76.919266


In [14]:
# also for kinetics, convert well indexes into substrate concentrations

def my_s( x ):
    my_ss = dict( zip( 'ABCDEFGH', [ .75, .1875, .046875, .01171875, .002929688, .000732422, .000183105, 0 ] ) )
    return my_ss[ x[0] ] 

kinetics[ 's' ] = kinetics.well.map( my_s ) 

In [15]:
import numpy as np 
np.mean( [ .75, .1875, .046875, .01171875, .002929688, .000732422, .000183105, 0 ] )

0.124992370625

In [16]:
# anything we need to manually drop? 

to_drop = [ 'WT', 'WT1' ] 

for df in [ kinetics, thermal ]:
    for item in to_drop:
        if item in df.index:
            df.drop( item, inplace=True ) 

In [17]:
# now export "cleaned" data 

kinetics.to_csv( '../pub/kinetics.csv' ) 
thermal.to_csv( '../pub/thermal.csv' ) 